## Sieć dla agenta do gry w Connect4 

In [45]:
import matplotlib.pyplot as plt
%matplotlib inline

import torch 
import torch.nn as nn
import torch.nn.functional as F

from torchsummary import summary

from imp import reload

import DataLoader
reload(DataLoader)

from DataLoader import InMemDataLoader
from DataLoader import C4DataSet

In [ ]:
nr_of_games = 1
moves_observed  = "all"

dataset = C4DataSet(nr_of_games, moves_observed).create_data_set(task_nr=2)

batch_size = 128
train_size, val_size, test_size = int(len(dataset)/3), int(len(dataset)/3), len(dataset)/3
amount_of_train_batches = train_size / batch_size

print(f"train_size = {train_size}")
train_set = dataset[:train_size]
val_set = dataset[train_size:train_size+val_size] 
test_set = dataset[train_size+val_size:]

data_loaders = {
    "train": InMemDataLoader(train_set, batch_size=batch_size, shuffle=True),
    "valid": InMemDataLoader(val_set, batch_size=batch_size, shuffle=False),
    "test": InMemDataLoader(test_set, batch_size=batch_size, shuffle=False),
}

In [56]:
import DataLoader
reload(DataLoader)
from DataLoader import C4DataSet

# game = 'S233322625223356466550404A' #<-- dobry przykłąd skosu, dziury 
# game = "S3256323326066002103304101B"
game = "S3410241441125533221302A"
dataset = C4DataSet(1, "all")
samples = dataset.test_samples(game)
for sample in samples:
    print(sample)

verti 3, 4
hole in 4, 2
hole in 3, 1
triple skos lewo 4, 4
triple skos lewo 4, 5
triple skos lewo 4, 5
triple skos lewo 3, 4
board =
 tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 2., 0., 0., 0., 0.],
        [0., 1., 2., 2., 1., 0., 0.],
        [0., 2., 1., 2., 2., 0., 0.],
        [1., 1., 2., 1., 2., 2., 0.],
        [2., 1., 1., 1., 2., 1., 0.]])
(tensor([0., 2., 0., 0., 0., 0., 0., 0.]), 1)
(tensor([0., 1., 0., 0., 0., 0., 0., 0.]), 1)
(tensor([0., 2., 0., 0., 0., 0., 0., 0.]), 1)
(tensor([0., 1., 0., 0., 0., 0., 0., 0.]), 1)
(tensor([0., 2., 0., 0., 0., 0., 0., 0.]), 1)
(tensor([0., 1., 0., 0., 0., 0., 0., 1.]), 1)
(tensor([0., 2., 0., 0., 0., 0., 0., 1.]), 1)
(tensor([0., 1., 0., 1., 0., 0., 0., 1.]), 1)
(tensor([0., 2., 0., 0., 0., 0., 0., 0.]), 1)
(tensor([0., 1., 0., 0., 0., 0., 0., 0.]), 1)
(tensor([0., 2., 0., 0., 0., 0., 0., 0.]), 1)
(tensor([0., 1., 0., 0., 0., 0., 0., 1.]), 1)
(tensor([0., 2., 0., 0., 0., 0., 0., 1.]), 1)
(tensor([0., 1., 0., 0., 0., 1., 0., 3.

In [13]:
print(f"rozmiar zbioru treningowego = {len(train_set)}")

for sample in test_set:
    print(sample)

# 33 12 11 56 55 10 35 33 65 51 20 66 02 20 23 40 4

rozmiar zbioru treningowego = 11
(tensor([0., 2., 0., 0.]), 0)
(tensor([0., 1., 0., 0.]), 0)
(tensor([0., 2., 0., 0.]), 0)
(tensor([0., 1., 0., 0.]), 0)
(tensor([0., 2., 1., 0.]), 0)
(tensor([0., 1., 1., 0.]), 0)
(tensor([0., 2., 1., 0.]), 0)
(tensor([0., 1., 1., 0.]), 0)
(tensor([0., 2., 1., 0.]), 0)
(tensor([0., 1., 1., 0.]), 0)
(tensor([1., 2., 1., 0.]), 0)


In [ ]:
import wandb
wandb.login() # klucz - 7242fe50822869937a282970b5385963778c7f8c

In [ ]:
class Model(nn.Module):
    def __init__(self, dp=0.5):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(2, 10, 4, padding=1)
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 5, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(5)
        self.fc1 = nn.Linear(5*5*6, 2)
        self.bn3 = nn.BatchNorm1d(3)
        self.do = nn.Dropout(0.45)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.conv2(x)
        x = F.relu(self.bn2(x))

        x = x.view(x.shape[0], -1)

        x = self.fc1(x)
        x = self.do(x) # dropout
        x = nn.Softmax()(x)
        return x

    def loss(self, Out, Targets):
      return F.cross_entropy(Out, Targets)

model = Model()
summary(model, (2, 6, 7))

In [132]:
def compute_error_rate(model, data_loader, device="cpu"):
  model.eval()

  num_errs, num_examples = 0, 0
  with torch.no_grad():
    for batch in data_loader:

      x, y = batch[0].to(device), batch[1].to(device)
      out = model(x)

      _, pred = out.max(dim=1)
      num_errs += (pred != y.data).sum().item()
      num_examples += x.size(0)

  return num_errs / num_examples

In [133]:
def train(num_of_epochs, train_loader, opt, print_every=10, device="cpu"):
  model.train()

  for data_loader in data_loaders.values():
    if isinstance(data_loader, InMemDataLoader):
        data_loader.to(device)

  iter = 0
  for e in range(num_of_epochs):
    model.train()
    print(f"Epoch {e+1}")

    for batch in train_loader:
      iter += 1

      x = batch[0].to(device)
      y = batch[1].to(device)
     
      opt.zero_grad()
      out = model(x)
      loss = nn.CrossEntropyLoss()(out, y)
      loss.backward()
      opt.step()

      _, pred = out.max(dim=1)
      batch_err = (pred != y).sum().item() / out.size(0)

      if iter % print_every == 0:
        print(f"iter = {iter}, batch_err = {batch_err * 100.0}")
        wandb.log({"batch_error_rate": batch_err})

    val_err = compute_error_rate(model, data_loader=data_loaders["valid"], device=device)
    print(f"val err = {100*val_err:.2f}")
    wandb.log({'val_err': val_err, 'epoch': e+1})

In [134]:
def initialize_weights(model):
    with torch.no_grad():
        for name, p in model.named_parameters():
            if 'weight' in name:
                if 'conv' in name:
                    f_in = p.shape[1]*p.shape[2]*p.shape[3]
                    p.normal_(0, torch.sqrt(torch.tensor(2./f_in)))
                elif 'bn' in name:
                    p = torch.ones_like(p)
                elif 'fc' in name:
                    f_in = p.shape[1]
                    p.normal_(0, torch.sqrt(torch.tensor(2./f_in)))
                else:
                    raise Exception('weird weight')

            elif 'bias' in name:
                p.zero_()
            else:
                raise Exception('weird parameter')

In [144]:
cnt = 0

In [145]:
lr = 0.0002
weight_decay = 0.000001
momentum = 0.9
epochs = 100
_device = "cpu"
_print_every = 20

opt = torch.optim.SGD(model.parameters(), lr=lr, weight_decay = weight_decay, momentum = momentum)
# opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [147]:
model = Model().to(device)
initialize_weights(model)

cnt +=  1
run_name = "run_nr " + str(cnt) + " simple cnn network (without dropouts) 100 games" 
run = wandb.init(
    project="Assigmnent4",
    name = run_name,
    config={
        "epochs": epochs,
        "learning_rate": lr,
        "optimizer": "SGD",
        "momentum": momentum,
        "weight_decay": weight_decay,
        "batch & dropout": True,
    },
)
wandb.watch(model, F.cross_entropy, log="all")

train(num_of_epochs=epochs, train_loader=data_loaders["train"], opt=opt, print_every=_print_every, device=_device)
test_error_rate = compute_error_rate(model, data_loaders["test"], device=_device)
wandb.finish()

#  TO DO LIST: (do pierwszego zadania)
# (0) ZAKODOWAĆ W PLANSZY kto ma ruch
# (3) dodać dropout po kążdej warstwie 

# a w drugi sięc bez konwolucji (dużo cech: np. czyj ruch )

Epoch 1
val err = 34.32
Epoch 2
val err = 34.23
Epoch 3
iter = 20, batch_err = 40.625
val err = 34.14
Epoch 4
val err = 34.14
Epoch 5
iter = 40, batch_err = 46.875
val err = 34.14
Epoch 6


/home/michal/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


val err = 34.14
Epoch 7
iter = 60, batch_err = 47.65625
val err = 34.14
Epoch 8
val err = 34.14
Epoch 9
iter = 80, batch_err = 48.4375
val err = 34.14
Epoch 10
val err = 34.14
Epoch 11
val err = 34.14
Epoch 12
iter = 100, batch_err = 36.71875
val err = 34.14
Epoch 13
val err = 34.14
Epoch 14
iter = 120, batch_err = 42.96875
val err = 34.14
Epoch 15
val err = 34.14
Epoch 16
iter = 140, batch_err = 44.53125
val err = 34.23
Epoch 17
val err = 34.14
Epoch 18
iter = 160, batch_err = 49.21875
val err = 34.14
Epoch 19
val err = 34.14
Epoch 20
iter = 180, batch_err = 40.74074074074074
val err = 34.23
Epoch 21
val err = 34.14
Epoch 22
val err = 34.23
Epoch 23
iter = 200, batch_err = 47.65625
val err = 34.14
Epoch 24
val err = 34.14
Epoch 25
iter = 220, batch_err = 50.0
val err = 34.14
Epoch 26
val err = 34.14
Epoch 27
iter = 240, batch_err = 50.78125
val err = 34.14
Epoch 28
val err = 34.14
Epoch 29
iter = 260, batch_err = 52.34375
val err = 34.14
Epoch 30
val err = 34.14
Epoch 31
val err = 34.

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


batch_error_rate,▃▅▅▆▁▃▄▆▅▆▆▇▅▅▅█▅▆▃▇▅▅▂▄▁▃▅▄▇▅▅▇█▅▆▂▇▆▆▄
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_err,█▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▁▁▁▁▁▁▁▁▁
batch_error_rate,0.44444
epoch,100
val_err,0.34137


In [ ]:
for p in model.parameters():
    p.requires_grad = False

model.eval()
with torch.no_grad():
    for x, y in data_loaders['train']:
        x = x.to(device)
        print(f"x = {x.shape}")
        # print(f"y = {y}")
        out = model(x)
        print(f"out = {out}")
        _, pred = out.max(dim=1)
        print(f"pred ={pred}")

        batch_err = (pred != y).sum().item() / out.size(0)

        print(batch_err)

